**Import Libraries**

In [20]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if geopy already used
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if folium already used
import folium # map rendering library

#!conda install -c conda-forge beautifulsoup4 --yes
#from bs4 import BeautifulSoup

print('Libraries imported.')
%matplotlib inline

Libraries imported.


**Import Sweden Cities**

In [36]:
import pandas as pd
path = 'https://raw.githubusercontent.com/laruchelle12/Data-Science/master/Sweden_Cities.csv' #csv file from https://www.latlong.net/category/cities-215-15.html
data = pd.read_csv(path, error_bad_lines=False)
data.head()

,City,Latitude,Longitude
0,Lulea,65.584816,22.156704
1,Trollhattan,58.283489,12.285821
2,Vasteras,59.611366,16.545025
3,Umea,63.825848,20.263035
4,Norrkoping,58.588455,16.188313


**Plot Cities in Sweden on Map**

In [37]:
address = 'Sweden'

geolocator = Nominatim(user_agent="sweden")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Sweden are {}, {}.'.format(latitude, longitude))

The coordinates of Sweden are 59.6749712, 14.5208584.


In [38]:
# create map of Sweden using latitude and longitude values
map_sweden = folium.Map(location=[latitude, longitude], zoom_start=5)

# add markers to map
for lat, lng, label in zip(data['Latitude'], data['Longitude'], data['City']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sweden)  
    
map_sweden

**Foursqaure Credentials**

In [39]:
CLIENT_ID = 'GBD2DH4LSOSVTHSC1AIGA042KAGWFFXUFYRLOKMP4RRXZSDC' # your Foursquare ID
CLIENT_SECRET = 'NNWZKHAATG5QVRA4NSM3WPBCQUZT2GQSF2STQZXCSJOZK1SX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GBD2DH4LSOSVTHSC1AIGA042KAGWFFXUFYRLOKMP4RRXZSDC
CLIENT_SECRET:NNWZKHAATG5QVRA4NSM3WPBCQUZT2GQSF2STQZXCSJOZK1SX


In [48]:
data.City

0          Lulea
1    Trollhattan
2       Vasteras
3           Umea
4     Norrkoping
5      Stockholm
6      Uddevalla
7      Vastervik
Name: City, dtype: object

In [50]:
City_latitude = data.Latitude # neighborhood latitude value
City_longitude = data.Longitude # neighborhood longitude value
City_name = data.City # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(City_name, 
                                                               City_latitude, 
                                                               City_longitude))

Latitude and longitude values of 0          Lulea
1    Trollhattan
2       Vasteras
3           Umea
4     Norrkoping
5      Stockholm
6      Uddevalla
7      Vastervik
Name: City, dtype: object are 0    65.584816
1    58.283489
2    59.611366
3    63.825848
4    58.588455
5    59.334591
6    58.351307
7    57.751442
Name: Latitude, dtype: float64, 0    22.156704
1    12.285821
2    16.545025
3    20.263035
4    16.188313
5    18.063240
6    11.885834
7    16.628838
Name: Longitude, dtype: float64.


**Gather 10 most common venues in City**

In [52]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500

In [53]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [55]:
sweden_venues = getNearbyVenues(names=data['City'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude'])

Lulea
Trollhattan
Vasteras
Umea
Norrkoping
Stockholm
Uddevalla
Vastervik


In [56]:
sweden_venues.head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lulea,65.584816,22.156704,Clarion Hotel Sense,65.585046,22.152880,Hotel
1,Lulea,65.584816,22.156704,Espresso House,65.583904,22.152566,Coffee Shop
2,Lulea,65.584816,22.156704,Kulturens Hus,65.585405,22.151082,Concert Hall
3,Lulea,65.584816,22.156704,Corsica,65.583498,22.149963,Italian Restaurant
4,Lulea,65.584816,22.156704,Waynes Coffee,65.584202,22.150951,Coffee Shop


In [57]:
sweden_venues.groupby('City').count().head()
print('There are {} uniques categories.'.format(len(sweden_venues['Venue Category'].unique())))


There are 99 uniques categories.


**One Hot Encoding**

In [58]:
# one hot encoding
sweden_onehot = pd.get_dummies(sweden_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sweden_onehot['City'] = sweden_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [sweden_onehot.columns[-1]] + list(sweden_onehot.columns[:-1])
sweden_onehot = sweden_onehot[fixed_columns]
sweden_onehot.head()

,City,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,BBQ Joint,Bakery,Bar,Basketball Stadium,Beer Bar,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Burger Joint,Bus Station,Café,Candy Store,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flower Shop,Food,Food Court,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Library,Liquor Store,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Multiplex,Museum,Music Venue,Nightclub,Opera House,Park,Pizza Place,Plaza,Pub,Restaurant,River,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shopping Mall,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Lulea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lulea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Lulea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Lulea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Lulea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [59]:
sweden_onehot.shape

(304, 100)

In [60]:
#Here we are showing the mean of what each city has close by
sweden_grouped = sweden_onehot.groupby('City').mean().reset_index()
sweden_grouped.head()

,City,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,BBQ Joint,Bakery,Bar,Basketball Stadium,Beer Bar,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Burger Joint,Bus Station,Café,Candy Store,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flower Shop,Food,Food Court,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Library,Liquor Store,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Multiplex,Museum,Music Venue,Nightclub,Opera House,Park,Pizza Place,Plaza,Pub,Restaurant,River,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shopping Mall,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Lulea,0.038462,0.0,0.00,0.00,0.000000,0.038462,0.0,0.00,0.0,0.038462,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.076923,0.00,0.038462,0.00,0.000000,0.000000,0.076923,0.000000,0.038462,0.0,0.00,0.00,0.038462,0.00,0.0,0.038462,0.00,0.076923,0.00,0.000000,0.00,0.00,0.00,0.00,0.0,0.000000,0.00,0.038462,0.038462,0.038462,0.00,0.076923,0.000000,0.000000,0.00,0.038462,0.00,0.00,0.00,0.00,0.000000,0.00,0.038462,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.00,0.00,0.000000,0.000000,0.000000,0.076923,0.00,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.038462,0.00,0.000000,0.00,0.038462,0.000000,0.0,0.00,0.00,0.00,0.038462,0.00
1,Norrkoping,0.023256,0.0,0.00,0.00,0.023256,0.000000,0.0,0.00,0.0,0.000000,0.023256,0.0,0.023256,0.000000,0.000000,0.023256,0.0,0.069767,0.00,0.000000,0.00,0.023256,0.023256,0.000000,0.000000,0.046512,0.0,0.00,0.00,0.000000,0.00,0.0,0.000000,0.00,0.023256,0.00,0.023256,0.00,0.00,0.00,0.00,0.0,0.023256,0.00,0.000000,0.000000,0.000000,0.00,0.046512,0.023256,0.000000,0.00,0.069767,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.023256,0.023256,0.0,0.000000,0.0,0.023256,0.023256,0.046512,0.023256,0.093023,0.000000,0.00,0.00,0.023256,0.046512,0.023256,0.069767,0.00,0.00,0.023256,0.00,0.0,0.023256,0.046512,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.00,0.00,0.000000,0.00
2,Stockholm,0.000000,0.0,0.02,0.01,0.000000,0.010000,0.0,0.03,0.0,0.000000,0.010000,0.0,0.010000,0.000000,0.000000,0.040000,0.0,0.050000,0.01,0.010000,0.01,0.030000,0.040000,0.010000,0.000000,0.010000,0.0,0.01,0.03,0.020000,0.01,0.0,0.000000,0.01,0.010000,0.01,0.000000,0.01,0.01,0.01,0.01,0.0,0.000000,0.02,0.000000,0.030000,0.000000,0.01,0.070000,0.010000,0.010000,0.01,0.020000,0.02,0.02,0.01,0.02,0.000000,0.01,0.000000,0.01,0.000000,0.000000,0.0,0.010000,0.0,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.01,0.03,0.000000,0.030000,0.010000,0.010000,0.01,0.01,0.000000,0.01,0.0,0.000000,0.030000,0.01,0.010000,0.000000,0.02,0.000000,0.01,0.000000,0.000000,0.0,0.01,0.01,0.02,0.000000,0.01
3,Trollhattan,0.026316,0.0,0.00,0.00,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.026316,0.026316,0.000000,0.0,0.078947,0.00,0.000000,0.00,0.000000,0.000000,0.026316,0.026316,0.000000,0.0,0.00,0.00,0.026316,0.00,0.0,0.000000,0.00,0.026316,0.00,0.000000,0.00,0.00,0.00,0.00,0.0,0.026316,0.00,0.000000,0.000000,0.026316,0.00,0.052632,0.000000,0.052632,0.00,0.000000,0.00,0.00,0.00,0.00,0.026316,0.00,0.000000,0.00,0.026316,0.000000,0.0,0.026316,0.0,0.000000,0.052632,0.052632,0.052632,0.052632,0.000000,0.00,

In [62]:
#Now, we are going to show what has more frequency in each of those city.
num_top_venues = 4

for hood in sweden_grouped['City']:
    print("----"+hood+"----")
    temp = sweden_grouped[sweden_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Lulea----
                  venue  freq
0         Shopping Mall  0.08
1                  Café  0.08
2                 Hotel  0.08
3  Fast Food Restaurant  0.08


----Norrkoping----
                venue  freq
0          Restaurant  0.09
1       Shopping Mall  0.07
2                Café  0.07
3  Italian Restaurant  0.07


----Stockholm----
          venue  freq
0         Hotel  0.07
1          Café  0.05
2  Cocktail Bar  0.04
3  Burger Joint  0.04


----Trollhattan----
              venue  freq
0  Sushi Restaurant  0.08
1              Café  0.08
2   Thai Restaurant  0.05
3  Tapas Restaurant  0.05


----Uddevalla----
                 venue  freq
0               Tunnel  0.67
1                  Gym  0.33
2  American Restaurant  0.00
3               Museum  0.00


----Umea----
                venue  freq
0               Hotel  0.11
1                Café  0.09
2  Italian Restaurant  0.06
3         Coffee Shop  0.06


----Vasteras----
        venue  freq
0        Café  0.09
1  Restaurant 

In [63]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [64]:
#And add it into a pandas dataframe to be able to apply afterwards Kclustering.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cities_sorted = pd.DataFrame(columns=columns)
cities_sorted['City'] = sweden_grouped['City']

for ind in np.arange(sweden_grouped.shape[0]):
    cities_sorted.iloc[ind, 1:] = return_most_common_venues(sweden_grouped.iloc[ind, :], num_top_venues)

cities_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Lulea,Café,Hotel,Fast Food Restaurant,Coffee Shop,Shopping Mall,American Restaurant,Gym / Fitness Center,Harbor / Marina,Wine Shop,Italian Restaurant
1,Norrkoping,Restaurant,Shopping Mall,Café,Italian Restaurant,Scandinavian Restaurant,Sushi Restaurant,Plaza,Concert Hall,Hotel,Pub
2,Stockholm,Hotel,Café,Cocktail Bar,Burger Joint,Deli / Bodega,Sushi Restaurant,Scandinavian Restaurant,Salad Place,Clothing Store,Bakery
3,Trollhattan,Café,Sushi Restaurant,Pub,Restaurant,Plaza,Pizza Place,Tapas Restaurant,Thai Restaurant,Scandinavian Restaurant,Indian Restaurant
4,Uddevalla,Tunnel,Gym,Yoga Studio,Flower Shop,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner


**K-Means Clustering**

In [2]:
# set number of clusters=5 (optimal)
kclusters = 5

sweden_grouped_clustering = sweden_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sweden_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

NameError: name 'sweden_grouped' is not defined

In [3]:
#distortions = []
#K = kmeans.labels_[0:10]
#for k in K:
    #kmeanModel = KMeans(n_clusters=k)
    #kmeanModel.fit(df)
    #distortions.append(kmeanModel.inertia_)

In [4]:
#plt.figure(figsize=(16,8))
#plt.plot(K, distortions, 'bx-')
#plt.xlabel('k')
#plt.ylabel('Distortion')
#plt.title('The Elbow Method showing the optimal k')
#plt.show()

In [96]:
# add clustering labels
#cities_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sweden_merged = data

# merge sweden_merged with cities_sorted to add latitude/longitude for each city
sweden_merged = sweden_merged.join(cities_sorted.set_index('City'), on='City')

sweden_merged

,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Lulea,65.584816,22.156704,3,Café,Hotel,Fast Food Restaurant,Coffee Shop,Shopping Mall,American Restaurant,Gym / Fitness Center,Harbor / Marina,Wine Shop,Italian Restaurant
1,Trollhattan,58.283489,12.285821,4,Café,Sushi Restaurant,Pub,Restaurant,Plaza,Pizza Place,Tapas Restaurant,Thai Restaurant,Scandinavian Restaurant,Indian Restaurant
2,Vasteras,59.611366,16.545025,1,Café,Restaurant,Hotel,Indian Restaurant,Asian Restaurant,Italian Restaurant,Bar,Shopping Mall,Coffee Shop,Pizza Place
3,Umea,63.825848,20.263035,1,Hotel,Café,Italian Restaurant,Coffee Shop,Bakery,Restaurant,Plaza,Pub,Diner,Convenience Store
4,Norrkoping,58.588455,16.188313,1,Restaurant,Shopping Mall,Café,Italian Restaurant,Scandinavian Restaurant,Sushi Restaurant,Plaza,Concert Hall,Hotel,Pub
5,Stockholm,59.334591,18.063240,1,Hotel,Café,Cocktail Bar,Burger Joint,Deli / Bodega,Sushi Restaurant,Scandinavian Restaurant,Salad Place,Clothing Store,Bakery
6,Uddevalla,58.351307,11.885834,0,Tunnel,Gym,Yoga Studio,Flower Shop,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
7,Vastervik,57.751442,16.628838,2,Flower Shop,Grocery Store,Department Store,Yoga Studio,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Falafel Restaurant


In [97]:
#Deletint the NaN Cluster Labels and making them int for ploting purposes in the future.
cities_sorted.shape
cities_sorted = cities_sorted.dropna(axis='rows')
cities_sorted['Cluster Labels'] = cities_sorted['Cluster Labels'].astype(int)
cities_sorted.head()

,Cluster Labels,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Lulea,Café,Hotel,Fast Food Restaurant,Coffee Shop,Shopping Mall,American Restaurant,Gym / Fitness Center,Harbor / Marina,Wine Shop,Italian Restaurant
1,1,Norrkoping,Restaurant,Shopping Mall,Café,Italian Restaurant,Scandinavian Restaurant,Sushi Restaurant,Plaza,Concert Hall,Hotel,Pub
2,1,Stockholm,Hotel,Café,Cocktail Bar,Burger Joint,Deli / Bodega,Sushi Restaurant,Scandinavian Restaurant,Salad Place,Clothing Store,Bakery
3,4,Trollhattan,Café,Sushi Restaurant,Pub,Restaurant,Plaza,Pizza Place,Tapas Restaurant,Thai Restaurant,Scandinavian Restaurant,Indian Restaurant
4,0,Uddevalla,Tunnel,Gym,Yoga Studio,Flower Shop,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner


In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sweden_merged['Latitude'], sweden_merged['Longitude'], sweden_merged['City'], cities_sorted['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [81]:
#Cluster 0 (Red color)
sweden_merged.loc[sweden_merged['Cluster Labels'] == 0, sweden_merged.columns[[0] + list(range(4, sweden_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Uddevalla,Tunnel,Gym,Yoga Studio,Flower Shop,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner


In [83]:
#Cluster 3 (Light Blue color)
sweden_merged.loc[sweden_merged['Cluster Labels'] == 3, sweden_merged.columns[[0] + list(range(4, sweden_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Lulea,Café,Hotel,Fast Food Restaurant,Coffee Shop,Shopping Mall,American Restaurant,Gym / Fitness Center,Harbor / Marina,Wine Shop,Italian Restaurant


In [84]:
#Cluster 4 (Green color)
sweden_merged.loc[sweden_merged['Cluster Labels'] == 4, sweden_merged.columns[[0] + list(range(4, sweden_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Trollhattan,Café,Sushi Restaurant,Pub,Restaurant,Plaza,Pizza Place,Tapas Restaurant,Thai Restaurant,Scandinavian Restaurant,Indian Restaurant


In [85]:
#Cluster 1 (Purple color)
sweden_merged.loc[sweden_merged['Cluster Labels'] == 1, sweden_merged.columns[[0] + list(range(4, sweden_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Vasteras,Café,Restaurant,Hotel,Indian Restaurant,Asian Restaurant,Italian Restaurant,Bar,Shopping Mall,Coffee Shop,Pizza Place
3,Umea,Hotel,Café,Italian Restaurant,Coffee Shop,Bakery,Restaurant,Plaza,Pub,Diner,Convenience Store
4,Norrkoping,Restaurant,Shopping Mall,Café,Italian Restaurant,Scandinavian Restaurant,Sushi Restaurant,Plaza,Concert Hall,Hotel,Pub
5,Stockholm,Hotel,Café,Cocktail Bar,Burger Joint,Deli / Bodega,Sushi Restaurant,Scandinavian Restaurant,Salad Place,Clothing Store,Bakery


In [86]:
#Cluster 2 (Orange color)
sweden_merged.loc[sweden_merged['Cluster Labels'] == 2, sweden_merged.columns[[0] + list(range(4, sweden_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Vastervik,Flower Shop,Grocery Store,Department Store,Yoga Studio,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Falafel Restaurant
